<div style="max-width:1400px;margin-center: auto">
<img src="images\race join.png" width="600"/>
</div>

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('race_join').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
races_df = spark.read.csv(r'E:\unused\Udemy\Spark_practice\raw\raw_files\races.csv', header =True, inferSchema =True).\
            withColumnRenamed('year','race_year').\
            withColumnRenamed('name','race_name').\
            withColumnRenamed('date','race_date')
constructors_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\processed\constructors').\
            withColumnRenamed('name','team')
drivers_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\processed\drivers').\
            withColumnRenamed('name','driver_name').\
            withColumnRenamed('number','driver_number').\
            withColumnRenamed('nationality','driver_nationality')
circuit_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\processed\circuit').\
                    withColumnRenamed('location','circuit_location')
results_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\processed\results').\
            withColumnRenamed('time','race_time')
races_df.show(2)
constructors_df.show(2)
drivers_df.show(2)
circuit_df.show(2)
results_df.show(2)

In [3]:
fnl_df = results_df.join(constructors_df, results_df.constructor_id == constructors_df.constructor_id,'inner').\
        join(drivers_df,results_df.driver_id == drivers_df.driver_id,'inner').\
        join(races_df,results_df.race_id == races_df.raceId,'inner').\
        join(circuit_df,races_df.circuitId == circuit_df.circuit_id,'inner').\
        select(races_df.race_year,
               races_df.race_name,
               races_df.race_date,
               circuit_df.circuit_location,
               drivers_df.driver_name,
               drivers_df.driver_number,
               drivers_df.driver_nationality,
               constructors_df.team,
               results_df.grid,
               results_df.fastest_lap,
               results_df.race_time,
               results_df.points,
              results_df.position)
fnl_df.show(2)

In [4]:
filtered = fnl_df.filter("race_year == 2020 and race_name == 'Abu Dhabi Grand Prix'").\
                orderBy(fnl_df.points.desc())
filtered.show(5)

In [5]:
fnl_df.write.parquet('presentation/race_result')